<a href="https://colab.research.google.com/github/JMML2021/0401_Registre-de-logs/blob/main/0401_Acceso_Expresiones_Regulares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing libraries
from urllib.request import urlretrieve
import pandas as pd
import os
import re
from datetime import datetime
import pytz
#import geocoder
import folium
import time
import urllib.request
import json
#import geopandas
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
# Activo Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Abro archivo

df = list(open('/content/drive/MyDrive/Web_access_log-akumenius.com.txt'))
file= '/content/drive/MyDrive/Web_access_log-akumenius.com.txt'
df[200:210]

['www.akumenius.com 180.76.5.31 - - [23/Feb/2014:03:15:03 +0100] "GET /destinos-baratos/destinos-caracteristicas/hoteles-baratos-en-Zurich_SUIZA-con-Alquiler-de-silla-de-ruedas HTTP/1.1" 200 8360 "-" "Mozilla/5.0 (compatible; Baiduspider/2.0; +http://www.baidu.com/search/spider.html)" VLOG=-\n',
 'www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:15:03 +0100] "GET /hoteles-baratos/ofertas-hotel-Capitolio-Apartamentos-Turisticos-en-Merida-207271b-destinos.html HTTP/1.1" 404 3100 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=-\n',
 'www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:15:06 +0100] "GET /destinos-baratos/destinosEstrelles/hoteles-en-Gary_ESTADOS%20UNIDOS-con-4-estrellas HTTP/1.1" 200 8812 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=-\n',
 'www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:15:06 +0100] "GET /hoteles-baratos/ofertas-hotel-Guitart-Termes-La-Collada-en-La%20Molina-6833b-destinos.

Una vez que se imprimen las líneas, vemos que están en las líneas muestra Formato de registro común (CLF):

mira aquí lo que significa CLF

sabiendo esto, mostremos la estructura del formato de registro:

%h %l %u %t "%r" %>s %b "%{Referente}i" "%{Usuario-agente}i\

dónde:

%h es la dirección IP del cliente (host remoto);

%l es la identidad RFC 1413 (no determinado);

%u es la identificación del usuario, determinada por la autenticación HTTP;

%t es la hora, por defecto, formateada como [día/mes/año:hora:minuto:segunda zona];

"%r" es la cadena de solicitud, formateada como "protocolo de recurso de método"

%>s es el código de estado;

%O es el tamaño de la solicitud;

"%{Referer}i" es el encabezado de solicitud HTTP de referencia y

"%{User-Agent}i" es el encabezado de solicitud HTTP de User-Agent.


In [15]:
def analizar_palabra(x):
    """
    Returns the string delimited by two characters.

    Example:
        `>>> parse_str('[string]')`
        `'string'`
    """
    return x[1:-1]

def analizar_fechaHora(x):
    '''
    Parses datetime with timezone formatted as:
        `[day/month/year:hour:minute:second zone]`

    Example:
        `>>> parse_datetime('13/Nov/2015:11:45:42 +0000')`
        `datetime.datetime(2015, 11, 3, 11, 45, 4, tzinfo=<UTC>)`

    Due to problems parsing the timezone (`%z`) with `datetime.strptime`, the
    timezone will be obtained using the `pytz` library.
    '''
    dt = datetime.strptime(x[1:-7], '%d/%b/%Y:%H:%M:%S')
    dt_tz = int(x[-6:-3])*60+int(x[-3:-1])
    return dt.replace(tzinfo=pytz.FixedOffset(dt_tz))

In [16]:
data = pd.read_csv(file,

    sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
    engine='python',
    na_values='-',
    header=None,
    usecols=[0, 1, 4, 5, 6, 7, 8, 9],
    names=['virtual_host','ip', 'time', 'request', 'status', 'size', 'referer', 'user_agent'],
    converters={'time': analizar_fechaHora,
                'request': analizar_palabra,
                'status': int,
                'size': int,
                'referer': analizar_palabra,
                'user_agent': analizar_palabra})

In [18]:
data.shape

(261873, 8)

In [17]:
data.sample(10)

,virtual_host,ip,time,request,status,size,referer,user_agent
195706,localhost,127.0.0.1,2014-02-27 13:07:49+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)
192980,www.akumenius.com,92.58.37.48,2014-02-27 12:15:27+01:00,POST /hotel-list-data/ HTTP/1.1,200,19104,http://www.akumenius.com/hotel-list,Mozilla/5.0 (iPad; CPU OS 7_0 like Mac OS X) A...
21737,www.akumenius.com,190.100.112.14,2014-02-23 15:49:49+01:00,GET /modules/raton/views/themes/bcoos/images/b...,200,7452,http://www.akumenius.com/destinos-caracteristi...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
227936,www.akumenius.com,157.55.33.122,2014-02-28 04:54:50+01:00,GET /escapadas/san-valentin-976/14.html HTTP/1.1,200,10773,NaN,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
246857,www.akumenius.com,66.249.64.98,2014-03-01 09:57:06+01:00,GET /destinosCaracteristicas/hoteles-baratos-e...,200,6393,NaN,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...
50454,www.akumenius.com,66.249.76.216,2014-02-24 09:28:01+01:00,GET /destinosCaracteristicas/hoteles-baratos-e...,200,7160,NaN,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...
225837,www.akumenius.com,66.249.76.216,2014-02-28 02:24:29+01:00,GET /destinosCaracteristicas/hoteles-baratos-e...,200,8033,NaN,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...
202198,www.akumenius.com,90.175.73.135,2014-02-27 15:32:27+01:00,POST /hotel-list-data/ HTTP/1.1,200,45713,http://www.akumenius.com/hotel-list,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
198797,www.akumenius.com,85.48.148.136,2014-02-27 15:01:32+01:00,GET /includes/css/style.css HTTP/1.1,200,3637,http://www.akumenius.com/,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:27.0) G...
116951,www.akumenius.com,195.245.193.60,2014-02-25 16:16:03+01:00,GET /modules/raton/views/themes/bcoos/css/2col...,200,11146,http://www.akumenius.com/?gclid=CKzmmIfJ57wCFY...,Mozilla/5.0 (iPhone; CPU iPhone OS 7_0 like Ma...
